# Template für Topic Modeling
Dieses Template soll dabei helfen, Topic Modeling automatisiert und einheitlich durchzuführen. 

## Allgemeine Vorbereitungsschritte

In [27]:
# Aktuelles Arbeitsverzeichnis anzeigen und bei Bedarf anpassen
# print(os.getcwd())
# os.chdir("C:/SV/HEX/Topic Modeling")

### Pakete laden

In [28]:
# Aktuelles Arbeitsverzeichnis anzeigen und bei Bedarf anpassen
import os
print(os.getcwd())
os.chdir("C:/Users/mhu/Documents/github/topic_model_it/")
#os.chdir("C:/Users/mhu/Documents/github/HEX_topic_model_it/")

import pandas as pd
import stopwords
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from umap import UMAP
from hdbscan import HDBSCAN
from sklearn.feature_extraction.text import CountVectorizer
import sklearn
import re
import spacy
import numpy as np
import random
import torch
from bertopic.vectorizers import ClassTfidfTransformer 
import openpyxl
import optuna
from sklearn.cluster import KMeans
from bertopic.representation import MaximalMarginalRelevance

model = SentenceTransformer("paraphrase-multilingual-mpnet-base-v2", device="cuda" if torch.cuda.is_available() else "cpu")
print("CUDA available:", torch.cuda.is_available())
print("Current device:", model.device)

C:\Users\mhu\Documents\github\topic_model_it
CUDA available: True
Current device: cuda:0


### Seed setzen
Wir setzen einen festen Seed, um Zufallszahlen in NumPy und PyTorch reproduzierbar zu machen, sowohl auf der CPU als auch auf der GPU (falls verfügbar). Das stellt sicher, dass Berechnungen mit zufälligen Operationen bei wiederholter Ausführung dieselben Ergebnisse liefern.

In [29]:
seed = 40  # Initialisiert den Seed-Wert für reproduzierbare Ergebnisse
np.random.seed(seed)  # Setzt den Seed für NumPy-Zufallszahlengeneratoren
random.seed(seed)  # Setzt den Seed für den Python-eigenen Zufallszahlengenerator
torch.manual_seed(seed)  # Setzt den Seed für PyTorch-Zufallszahlen
if torch.cuda.is_available():  # Überprüft, ob CUDA (GPU-Unterstützung) verfügbar ist
    torch.cuda.manual_seed_all(seed)  # Setzt den Seed für alle CUDA-Zufallszahlen (für GPU-Berechnungen)

### Datensätze einlesen
Der Trainings- und der Test-Datensatz werden hier eingelesen. Als Faustregel gilt, der Trainingsdatensatz sollte 80% und der Test-Datensatz 20% des Volumens ausmachen. 
Der Trainings-Datensatz wird für das trainieren / fitten des Modells verwendet. Der Test-Datensatz beinhaltet eine (in diesem Fall manuell erstellte) sogenannte "Ground Truth". Dies ist der Goldstandard, anhand dessen das Modell auf Performance hin überprüft wird. 

In [30]:
# Training-Datensatz
training_set = pd.read_csv("data/informatikkurse.csv")  # Liest die CSV-Datei ein und speichert sie in einem DataFrame
# training_set = training_set.sample(n=500, random_state=42)  # Zieht eine Zufallsstichprobe von 500 Zeilen aus dem DataFrame mit festgelegtem Seed für Reproduzierbarkeit
training_set = training_set.apply(lambda x: x.fillna('') if x.dtype == 'O' else x)  # Ersetzt fehlende Werte durch leere Strings in Objektspalten (Strings) und belässt numerische Spalten unverändert
training_set['titel_kursbesch'] = training_set['veranstaltung_titel'] + ' ' + training_set['kursbeschreibung']  # Kombiniert die Spalten "titel" und "kursbeschreibung" zu einer neuen Spalte "titel_kursbesch"
docs = training_set['titel_kursbesch'].tolist()  # Konvertiert die Inhalte der Spalte "titel_kursbesch" in eine Liste von Strings

C:\Users\mhu\AppData\Local\Temp\ipykernel_34904\3557994817.py:2: DtypeWarning: Columns (15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  training_set = pd.read_csv("data/informatikkurse.csv")  # Liest die CSV-Datei ein und speichert sie in einem DataFrame


In [31]:
import pandas as pd

# 1. Daten laden und NaNs bereinigen
df = pd.read_csv("data/labelled_data_final.csv", sep=";").fillna('')

# 2. Kombination und Vorab-Filterung (entfernt leere Synonyme)
df['titel_kursbesch'] = df['veranstaltung_titel'] + ' ' + df['kursbeschreibung']
df = df[df["Synonyme"].str.contains(r'[a-zA-Z0-9]')] # Behält nur Zeilen mit echten Zeichen (isalnum)

# 3. Ground Truth: In einem Rutsch splitten und bereinigen
# Wir nutzen .str.split mit Regex, um Whitespace um Kommas direkt zu entfernen
ground_truth = (
    df["Synonyme"]
    .str.lower()
    .str.split(r'\s*,\s*')
    .apply(lambda terms: [t for t in terms if t.strip()]) # Filtert leere Strings innerhalb der Liste
).tolist()

# 4. Finales Test-Set
test_set = df['titel_kursbesch'].reset_index(drop=True)

In [32]:
ground_truth

[['software development',
  'software design',
  'software architecture',
  'application development',
  'software engineering',
  'softwareentwicklung',
  'softwaredesign',
  'softwarearchitektur',
  'anwendungsentwicklung'],
 ['theory of computation',
  'computational theory',
  'formal methods',
  'automata theory',
  'theoretische informatik',
  'berechenbarkeitstheorie',
  'computertheorie',
  'formale methoden',
  'automatentheorie'],
 ['robot systems',
  'autonomous robots',
  'robot engineering',
  'robot programming',
  'robotik',
  'robotersysteme',
  'autonome roboter',
  'robotertechnik',
  'roboterprogrammierung'],
 ['computational biology',
  'genomic analysis',
  'biological data analysis',
  'bioinformatik',
  'computergestützte biologie',
  'genomanalyse',
  'biologische datenanalyse'],
 ['big data analytics',
  'data analytics',
  'data mining',
  'data engineering',
  'data science und big data',
  'big-data-analyse',
  'datenanalyse',
  'data mining',
  'datenengine

## NLP Vorbereitungsschritte
Zunächst werden die Trainingsdaten eingelesen und die gängigen Vorbereitungsschritte für NLP durchgeführt. Diese wären:
* Stopwords entfernen
* CountVectorizer spezifizieren

### Stopwords entfernen
Im Kontext des hier zu modellierenden Topic Modells werden sowohl standardisierte englische, deutsche als auch individuelle Stopwords generiert und im Objekt `sw` zusammengespielt.
Die Stopwords können je nach Anwendungsfall ergänzt oder reduziert werden.

In [33]:
from utils import stopwords_config

irrelevant_terms = stopwords_config.irrelevant_terms

sw = list(stopwords.get_stopwords("en"))
sw.extend(list(stopwords.get_stopwords("de")))
sw.extend(irrelevant_terms) 

### Lemmatisierung
Durch Lemmatisierung werden die Wörter in einheitliche Begriffe umgewandelt, sodass diese robuster werden. 

## Anwendung: Konfiguration, Training und Evaluation des Topic Models
Hier muss alles innerhalb einer einzigen Code-Zelle erfolgen, da bei allen Konfigurationen variable Parameter vorkommen und wir diese durch das optuna-Package optimieren wollen.

In [39]:
# Im Objective werden die verschiedenen Parameter-Settings gesetzt, über welche man optimieren möchte
def objective(trial):

  try:
    # Embedding Settings
    embedding_model_name = trial.suggest_categorical("embedding_model", ["paraphrase-multilingual-mpnet-base-v2", "paraphrase-multilingual-MiniLM-L12-v2"])
    # UMAP Settings
    n_neighbors = trial.suggest_categorical("n_neighbors", [15, 30, 50])
    min_dist = 0.0
    n_components = trial.suggest_categorical("n_components", [5, 7, 10])
    # HDBSCAN Settings
    min_cluster_size = trial.suggest_int("min_cluster_size", 10, 100, 10) # Statt [80, 320, 480]    #min_samples = trial.suggest_int("min_samples", 3, 10, 1)
    #min_samples = trial.suggest_int("min_samples", 3, 10, 1)
    # BERTopic Settings
    nr_topics = trial.suggest_int("nr_topics", 20, 50, 10)
    diversity = trial.suggest_float("diversity", 0.0, 0.2, step = 0.1)
    min_topic_size = trial.suggest_int("min_topic_size", 80, 100, 10)


    #--------------------------------------------------------------------------------------------------------------------------------------------
    # Konfiguration
    #--------------------------------------------------------------------------------------------------------------------------------------------

    # CountVectorizer
    vectorizer = CountVectorizer(
      stop_words=irrelevant_terms,  # Entfernt Stopwörter basierend auf der angegebenen Liste (irrelevant_terms)
      token_pattern=r'\b\w+\b',  # Extrahiert nur ganze Wörter, d. h. keine Sonderzeichen oder Zahlen
      ngram_range=(1, 3)  # Erstellt 1-Gramme (einzelne Wörter) bis 3-Gramme (Wortgruppen aus bis zu 3 aufeinanderfolgenden Wörtern)
    )

    # Embedding Settings  
    embedding_model = SentenceTransformer(embedding_model_name)
    
    # UMAP Settings
    umap_model = UMAP(n_neighbors=n_neighbors, min_dist=min_dist, n_components=n_components, metric="cosine", random_state=13)

    # HDBSCAN Settings
    hdbscan_model = HDBSCAN(min_cluster_size=min_cluster_size, gen_min_span_tree=True, prediction_data=True)

    # Representation Settings
    representation_model = MaximalMarginalRelevance(diversity=diversity)


    #--------------------------------------------------------------------------------------------------------------------------------------------
    # Training
    #--------------------------------------------------------------------------------------------------------------------------------------------

    # BERTopic initialisieren
    topic_model = BERTopic(
      embedding_model=embedding_model,
      min_topic_size=min_topic_size,
      nr_topics=nr_topics, 
      language="multilingual",
      umap_model=umap_model,
      vectorizer_model=vectorizer,
      hdbscan_model=hdbscan_model,
      top_n_words = 30,
      representation_model=representation_model
    )

    # BERTopic trainieren
    topic_model_quanten = topic_model.fit(docs)


    #--------------------------------------------------------------------------------------------------------------------------------------------
    # Evaluation
    #--------------------------------------------------------------------------------------------------------------------------------------------

    # BERTopic auf Test-Daten anwenden
    topics, probs = topic_model_quanten.transform(test_set)
    print(topic_model_quanten.get_topic_freq())

    # Outlier reduzieren
    topics = topic_model_quanten.reduce_outliers(test_set, topics)

    # Resultierende Topic-Nummern mit den Representations (= relevante Begriffe) zu einem Datensatz kombinieren
    dataframe_with_results_left = pd.DataFrame(topics, columns = ["Topic"])
    dataframe_with_results_right = pd.DataFrame(topic_model_quanten.get_topic_info().set_index('Topic')[['Representation']])
    dataframe_with_results = dataframe_with_results_left.join(dataframe_with_results_right, on="Topic")

    # Goldstandard (Ground Truth) mit den Ergebnissen abgleichen und Score berechnen (Score = Anteil korrekter Topic-Zuweisungen)
    row_number = 0
    metric = 0
    while row_number < len(ground_truth):
      # Da ground_truth bereits eine Liste von Listen ist:
      ground_truth_current_iteration = ground_truth[row_number]
      result_current_iteration = dataframe_with_results.at[row_number, "Representation"]

      # Überprüfen, ob irgendein Begriff aus dem Resultat im Goldstandard zum Text vorkommt (1 = ja, 0 = nein)
      if any(element in result_current_iteration for element in ground_truth_current_iteration):
              metric += 1
      else: metric += 0

      row_number = row_number+1

      print("TM", result_current_iteration)
      print("Ground Truth:", ground_truth_current_iteration)
      print("---------------------------------------------------------------------------------------------------------------")

    metric_score = metric/row_number


    #--------------------------------------------------------------------------------------------------------------------------------------------
    # Ergebnis printen und Score returnen
    #--------------------------------------------------------------------------------------------------------------------------------------------

    # Ergebnis printen
    #print("Modell evaluiert mit einem Score von ", metric_score, ". " \
    #"Verwendete Parameter: embedding model: ", embedding_model_name, ", nr_topics: ", nr_topics, ", n_neighbors: ", n_neighbors, ", min_dist: ", min_dist,
    #", n_components: ", n_components, ", min_cluster_size: ", min_cluster_size)
  
    return metric_score 
  
  except Exception as e:
      print("Trial wird aufgrund eines Errors übersprungen")
      print("Verwendete Parameter: embedding model: ", embedding_model_name, ", n_neighbors: ", n_neighbors, ", min_dist: ", min_dist,\
      ", n_components: ", n_components, ", min_cluster_size: ", min_cluster_size)
      print(e)
      raise optuna.TrialPruned()

In [37]:
study = optuna.create_study(direction = "maximize")
study.optimize(objective, n_trials = 100)

print("Best parameters:", study.best_params)

[I 2026-02-24 16:59:27,598] A new study created in memory with name: no-name-3e59b7d0-b91b-47e3-b885-9fb324497768
[I 2026-02-24 17:01:13,540] Trial 0 finished with value: 0.04671857619577308 and parameters: {'embedding_model': 'paraphrase-multilingual-mpnet-base-v2', 'n_neighbors': 50, 'n_components': 5, 'min_cluster_size': 10, 'nr_topics': 20, 'diversity': 0.1}. Best is trial 0 with value: 0.04671857619577308.


    Topic  Count
0       0  35010
1      -1   8336
7       1   2955
5       2   1770
4       3   1163
10      4    637
2       5    297
11      6    239
6       7    219
8       8    209
12      9     84
14     10     74
15     11     65
16     12     49
13     13     43
18     14     41
17     15     16
3      16     15
19     17     15
9      18     11
TM ['informatik', 'data', 'learning', 'systems', 'software', 'algorithmen', 'systeme', 'programmierung', 'methoden', 'engineering']
Ground Truth: ['software development', 'software design', 'software architecture', 'application development', 'software engineering', 'softwareentwicklung', 'softwaredesign', 'softwarearchitektur', 'anwendungsentwicklung']
---------------------------------------------------------------------------------------------------------------
TM ['informatik', 'data', 'learning', 'systems', 'software', 'algorithmen', 'systeme', 'programmierung', 'methoden', 'engineering']
Ground Truth: ['theory of computation', 'com

[I 2026-02-24 17:02:18,446] Trial 1 finished with value: 0.2347052280311457 and parameters: {'embedding_model': 'paraphrase-multilingual-MiniLM-L12-v2', 'n_neighbors': 15, 'n_components': 5, 'min_cluster_size': 80, 'nr_topics': 50, 'diversity': 0.2}. Best is trial 1 with value: 0.2347052280311457.


    Topic  Count
0      -1  28615
8       0   2101
9       1   1407
4       2   1345
22      3   1305
1       4   1219
5       5   1192
17      6   1189
11      7   1173
3       8   1102
12      9    951
6      10    925
24     11    904
20     12    644
21     13    520
13     14    443
16     15    386
14     16    349
30     17    338
35     18    337
7      19    300
36     20    290
31     21    276
27     22    261
18     23    250
43     24    236
15     25    221
34     26    219
25     27    186
41     28    186
10     29    171
45     30    162
23     31    161
19     32    148
38     33    147
49     34    135
40     35    130
39     36    130
47     37    129
32     38    128
26     39    121
29     40    107
33     41    102
46     42     91
44     43     88
2      44     87
42     45     87
37     46     87
48     47     84
28     48     83
TM ['software engineering', 'softwaretechnik', 'softwareprojekt', 'engineering software', 'software engineering software', 'engineeri

[W 2026-02-24 17:02:49,372] Trial 2 failed with parameters: {'embedding_model': 'paraphrase-multilingual-MiniLM-L12-v2', 'n_neighbors': 30, 'n_components': 7, 'min_cluster_size': 90, 'nr_topics': 20, 'diversity': 0.2} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\Users\mhu\miniconda3\envs\bertopic-env\lib\site-packages\optuna\study\_optimize.py", line 201, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\mhu\AppData\Local\Temp\ipykernel_34904\1842958600.py", line 61, in objective
    topic_model_quanten = topic_model.fit(docs)
  File "c:\Users\mhu\miniconda3\envs\bertopic-env\lib\site-packages\bertopic\_bertopic.py", line 387, in fit
    self.fit_transform(documents=documents, embeddings=embeddings, y=y, images=images)
  File "c:\Users\mhu\miniconda3\envs\bertopic-env\lib\site-packages\bertopic\_bertopic.py", line 472, in fit_transform
    umap_embeddings = self._reduce_dimensionality(embeddings, y)
  File "c:\Users\m

KeyboardInterrupt: 

In [ ]:
print("Best parameters:", study.best_params)